> ## Исследование надежности заемщиков

На основе статистики о платёжеспособности клиентов исследую влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

<b> Описание задачи
    
На основе данных кредитного отдела банка исследую платёжеспособность клиентов. Произведена предобработка данных: определены и обработаны пропуски, заменены типы данных на соответствующие, удалены дубликаты, также данные категоризованы. Один датафрейм декомпозирован на три.

<b> Вывод
    
<b> Лучше всего отдают кредиты на операции с недвижимостью и на проведение свадьбы, а на покупку автомобиля или на учебу - хуже. Хуже всего с выплатой кредитов у категории с минимальной и максимальной доходностью. Низкие показатели выплаты кредита в срок у неженатых и холостых и у пар, находящихся в гражданском браке. Также рост просрочки кредита увеличивается с ростом количества детей в семье.

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

In [2]:
data.head(20) # просмотрю файл с исходными данными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Предобработка данных

In [3]:
data.isna().sum() # Вывежу количество пропущенных значений для каждого столбца

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<b> В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце обработаю на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [4]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()
# с помощью цикла заполняю пропуски в столбце 'total_income' 
# значением на основе уникальных значений в столбце 'income_type'. 
# Если условие выполняется, то значение в столбце 'total_income' заменяется на медианное значение

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаю значения в этом столбце: заменяю все отрицательные значения положительными с помощью метода abs().

In [5]:
data['days_employed'] = data['days_employed'].abs() 
# меняю отрицательные значения в столбце 'days_employed' на положительные

In [6]:
data.groupby('income_type')['days_employed'].agg('median') 
# с помощью метода median() вывожу медианное значение трудового стажа для каждого типа занятости

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставляю их как есть. Тем более этот столбец не понадобится вам для исследования.

In [7]:
data['children'].unique() # вывожу уникальные значения

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

<b> В столбце children есть два аномальных значения. Удаляю строки, в которых встречаются такие аномальные значения.

In [8]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
# удаляю те строки, где значение в столбце 'children' равно -1 и равно 20
# (`!=` означает не равно)

In [9]:
data['children'].unique() # проверка

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [11]:
# заполняю пропуски в столбце 'days_employed' медианным значением по каждому типу из столбца 'income_type'
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum() # проверка пропущенных значений

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [13]:
# меняю тип данных в столбце 'total_income' с помощью метода astype() на целочисленные
data['total_income'] = data['total_income'].astype(int) 

В столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 
Приведите их к нижнему регистру

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
# вывожу количество строк дубликатов комбинацие методов duplicated() и sum()
data.duplicated().sum()

71

In [16]:
# удаляю строки дубликатов
data = data.drop_duplicates()

## Категоризация данных

<b> На основании диапазонов, указанных ниже, создаю в датафрейме data столбец total_income_category с категориями:

+ 0–30000 — 'E';
+ 30001–50000 — 'D';
+ 50001–200000 — 'C';
+ 200001–1000000 — 'B';
+ 1000001 и выше — 'A'

In [18]:
def categorize_income(income):
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'

In [20]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
# Метод apply() используется для применения функции к каждой строке или столбцу датафрейма

In [21]:
data['purpose'].unique() 
# посмотрю цели взятия кредитов из столбца 'purpose' с помощью метода unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [22]:
# создам цели для взятия кредита в столбец 'purpose_category' на категории
def categorize_purpose(row):
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
        return 'нет категории'

In [23]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [24]:
data.head(10) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [31]:
data_pivot_children=data.pivot_table(index='children', values='debt', aggfunc={'count', 'sum', 'mean'})

data_pivot_children

,count,mean,sum
children,,,
0,14091,0.075438,1063
1,4808,0.092346,444
2,2052,0.094542,194
3,330,0.081818,27
4,41,0.097561,4
5,9,0.000000,0


<b>_Вывод:_
    
  С ростом количества детей в семьях уменьшается доля возврата кредитов в срок.

In [32]:
data_pivot_family_status=data.pivot_table(index='family_status', values='debt', aggfunc={'count', 'sum', 'mean'})

data_pivot_family_status

,count,mean,sum
family_status,,,
Не женат / не замужем,2796,0.097639,273
в разводе,1189,0.070648,84
вдовец / вдова,951,0.066246,63
гражданский брак,4134,0.093130,385
женат / замужем,12261,0.075606,927


<b>_Вывод:_
    
Самый высокий показатель невозврата кредита в срок у свободных и находящихся в гражданском браке.

In [35]:
data_pivot_total_income_category=data.pivot_table(index='total_income_category', values='debt', aggfunc={'count', 'sum', 'mean'})

data_pivot_total_income_category

,count,mean,sum
total_income_category,,,
A,25,0.080000,2
B,5014,0.070602,354
C,15921,0.084982,1353
D,349,0.060172,21
E,22,0.090909,2


<b>_Вывод:_
    
    Хуже всего с возвратом кредита в срок у категории с доходом до 30000 рублей в месяц. Самыми объемными группами являются семьи с доходом 50001–200000 — 'C' и 200001–1000000 — 'B'. И по таблице иы видим, что чем выше доход, тем легче семьям планировать свои расходы и не задерживать выплату кредит

In [36]:
data_pivot_purpose_category=data.pivot_table(index='purpose_category', values='debt', aggfunc={'count', 'sum', 'mean'})

data_pivot_purpose_category

,count,mean,sum
purpose_category,,,
операции с автомобилем,4279,0.093480,400
операции с недвижимостью,10751,0.072551,780
получение образования,3988,0.092528,369
проведение свадьбы,2313,0.079118,183


<b>_Вывод:_
    
  Кредиты, взятые на покупку автомобиля и на получение образования реже всего отдают в срок.  

## Общий вывод

Нашёл ошибки и исправили их.

Я исследовал следующие параметры, влияющие на сроки погашения кредита: -влияние количества детей, -влияние семейного положения, -влияние дохода семьи, -влияние цели кредита.

Закончив расчеты можно говорить о том что:

Лучше всего отдают кредиты на операции с недвижимостью и на проведение свадьбы, а на покупку автомобиля или на учебу - реже. Хуже всего с выплатой кредитов у категории с минимальной и максимальной доходностью. Низкие показатели выплаты кредита в срок у неженатых и холостых и у пар, находящихся в гражданском браке. Также рост просрочки кредита увеличивается с ростом количества детей в семье.